This is a general purpose image classifier that can be used for most image classification problems.
No knowledge of neural networks or Tensorflow is required to use it.
An example of use on the Autism data set is shown below 
source_dir=r'c:\Temp\autism' # note use of r for raw and use a  \ not a/
output_dir=r'c:\Temp\autism'
subject='autism'
t_split=5
v_split=5 
epochs=30 
batch_size=80 
lr_rate=.0025 
lr_factor=.8
image_size=128 
rand_seed=256
model_size='L'
mode='sep'

TF2_classify(source_dir,output_dir,mode,subject, t_split=t_split, v_split=v_split, epochs=epochs,batch_size=batch_size, lr_rate=lr_rate,lr_factor,image_size=image_size,rand_seed=rand_seed, model_size=model_size)

The program operates in one of two modes, If mode-'all' the training, test and validation files
are taken from the source_dir and split into train, test and validation files as defined by 
t_split and v_split integer percentages. If model='sep' images are read in from the train, test and 
valid directories within the source directory. t-split and v_split values are not used in this mode.
epochs is the number of training epochs. lr_rate is a float defining the learning rate. 
lr_factor is a float. The learning rate is reduced by this factor each time an epoch results in a
validation loss larger than the lowest value thus far achieved. batch size is the number of images processed as a group during training. 
The program has 3 models. model_size='L' is a large model based on the MobileNet architecture. It is accurate but propcessing time and memory requirements can be large. For this model set batch_size-80. If you get a resource exhaust error reduce its value. If model_size="M' the program uses a medium sized model. Execution is fairly fast but it is less accurate. A batch size of 80 works well. If model_size="S" a small model is used. Execution is fast but accuracy is reduced. 
rand_seed sets the seed for the random generators. It's value is arbitrary but when changed will give a different mix of training, test and validation files when mode='all'.
image_size is the size that images are converted to for processing. If mode="L" image size is set internally at 224. subject is a string you can use to denote the name of files that are stored in your output_dir at the conclusion of the program. At the conclusion of training, the best saved model is loaded from the output_dir and the test set is processed. Test results are displayed if there are less than 25 errors. The error list is saved to a file in the output_dir.  At the conclusion of the program two files are stored in the output_dir. One of them is the resulting trained model file. It is labelled as subject-image_size-accuracy.h5 , For example autism-224-95.35.h5 means the subject was autism, the image size was 224 X 224 and the accuracy on the test set was 95.35%. This model file can then be used with a prediction program. The other file saved is a text file that can be easily converted into a python dictionary. The key is the class number and the value is the associated class name. It is labelled as subject.txt. This file will also be needed by a prediction program to generate a list of classes. The error list is stored in the output_dir as error-list-M-accuracy.txt where M is the model type.
After the test list is processed the program evaluates the accuray. If it is above 95% the program terminates. If the accuracy is below 95 precent the program will train for 10 more epochs starting from the last epoch. After these 10 epochs complete the new saved model is loaded and the test set is run again. Results are then shown after which the program terminates.
To use the program you need a python 3 environment in which you have loaded the modules tensorflow 2.0, numpy, matplot, sklearn, tqdm cv2,random and PIL.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import time
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
import os
from tqdm import tqdm
import random as rd

In [2]:
def get_data_sets(dir,output_dir,mode,subject,image_size,t_split, v_split, rand_seed ):
    data=[]
    labels=[]
    class_list=[]
    files=[]
    net_split=(t_split+ v_split)/100
    v_share=t_split/(t_split + v_split)
    t_split=t_split/100
    v_split=v_split/100
    if mode !='all':
        d_path=os.path.join(dir, "train")
    #    print(d_path)
    else:
        d_path=dir    # if all=True read all data from dir = source_dir, otherwise dir=source_dir/train 
    source_list=os.listdir(d_path)
    for s in source_list:
        s_path=os.path.join(d_path,s)
        if os.path.isdir(s_path):  # only process  directories not files
            class_list.append(s)
    class_count=len(class_list)  #determine number of class directories in order to set leave value intqdm    
    if mode != 'all':    # read data from  train, test and valid directories
        train_path=os.path.join(dir, 'train')
        tr_data=read_files(train_path,class_list,image_size,mode)
        # tr_data[0]=training files, tr_data[1]=training labels
       
        for i in range(0,3):
            rd.seed(rand_seed)
            rd.shuffle(tr_data[i]) #shuffle training data but keep the labels tied to the array by using same seed       
        test_path=os.path.join(dir,'test')
        t_data=read_files(test_path, class_list,image_size,mode)
        #t_data[0] test files, t_data[1]=test labels
        valid_path=os.path.join(dir,'valid')
        v_data=read_files(valid_path, class_list,image_size,mode)
        #shuffle the validation images but keep labels tied to the file by using same random seed
        for i in range(0,3):
            rd.seed(rand_seed)
            rd.shuffle(v_data[i])
        train_data=np.array(tr_data[0])
        train_labels=np.array(tr_data[1])
        train_files=np.array(tr_data[2])
        val_data=np.array(v_data[0])
        val_labels=np.array(v_data[1])
        val_files=np.array(v_data[2]) 
        test_data=np.array(t_data[0])
        test_labels=np.array(t_data[1])
        test_files=np.array(t_data[2])
        print(tr_data[1])
    else:  # all the data is in a single file,  split into train, test and validate
        data=read_files(dir, class_list,image_size,mode) # returns 3 lists, list of image arrays, image labels and image file names
        train_data , x , train_labels , y = train_test_split(data[0],data[1],test_size =net_split,random_state =rand_seed)
        train_files, a =train_test_split(data[2], test_size=net_split, random_state=rand_seed)
        val_data , test_data , val_labels , test_labels = train_test_split(x,y,test_size = v_share,random_state = rand_seed)
        val_files, test_files=train_test_split(a,test_size=v_share, random_state=rand_seed)        
        train_data=np.array(train_data)
        train_labels-np.array(train_labels)
        train_files=np.array(train_files)
        test_data=np.array(test_data)
        test_labels=np.array(test_labels)
        test_files=np.array(test_files)
        val_data=np.array(val_data)
        val_labels=np.array(val_labels)
        val_files=np.array(val_files)
    print_data(train_labels, test_labels, val_labels, class_list)
    # save the class dictionary as a text file so it can be used by classification.py in the future
    msg=''
    for i in range(0, class_count):
        msg=msg + str(i) + ':' + class_list[i] +','
    id=subject  + '.txt'   
    dict_path=os.path.join (output_dir, id)
    print('\n saving dictionary of class names and labels to {0}'.format(dict_path))
    f=open(dict_path, 'w')
    f.write(msg)
    f.close()
      
    return [train_data, train_labels, test_data, test_labels, val_data, val_labels,test_files, class_list]
   

In [19]:
def read_files(dir_path, class_list, image_size,mode):
    data=[]
    labels=[]
    files=[]
    if(len(class_list)<=6):
        leave=True
    else:
        leave=False    
    dir_list=os.listdir(dir_path)    #get list of items in dir
    class_count=-1
    for d in dir_list:   # these should be the class directories
        d_path=os.path.join(dir_path,d)  # path to class directories
        if os.path.isdir(d_path):  #only process directories ignore any files in dir
            d_list=d_path.split(r'/')
            length=len(d_list)
            desc=d_list[length-2] + '-' +d_list[length-1]
            class_count=class_count +1
            file_list=os.listdir(d_path)  #list contents of directory d , they should just be files but better to check
            for f in tqdm(file_list, desc=desc, unit='files', leave=leave):
                file_path=os.path.join(d_path,f)
                if os.path.isfile(file_path):  # just process files
                    index=f.rfind('.')
                    ext=f[index+1:].lower()  # get the file's extension
                    if ext in ['jpg', 'jpeg','jpe','jp2','tiff', 'png']:  # make sure image formats work with cv2
                        img=cv2.imread(file_path,1)
                        print(file_path)
                        image_array = Image.fromarray(img , 'RGB')
                        resize_img = image_array.resize((image_size ,image_size))
                        data.append(np.array(resize_img))   #data is a list of image arrays for the entire data set
                        labels.append(class_count) # class count is the label associated with the array image
                        files.append(f)
        
    return [data,labels,files]

In [4]:
def print_data(train_labels, test_labels, val_labels, class_list):
    train_list=list(train_labels)
    test_list=list(test_labels)
    val_list=list(val_labels)
    print('{0:9s}Class Name{0:10s}Class No.{0:4s}Train Files{0:7s}Test Files{0:5s}Valid Files'.format(' '))
    for i in range(0, len(class_list)):
        c_name=class_list[i]
        tr_count=train_list.count(i)
        tf_count=test_list.count(i)
        v_count=val_list.count(i)
        print('{0}{1:^25s}{0:5s}{2:3.0f}{0:9s}{3:4.0f}{0:15s}{4:^4.0f}{0:12s}{5:^3.0f}'.format(' ',
                                                                                               c_name,i,tr_count,
                                                                                               tf_count,v_count))
    print('{0:30s} ______________________________________________________'.format(' '))
    msg='{0:10s}{1:6s}{0:16s}{2:^3.0f}{0:8s}{3:3.0f}{0:15s}{4:3.0f}{0:13s}{5}\n'
    print(msg.format(' ', 'Totals',len(class_list),len(train_labels),len(test_labels),len(val_labels)))

In [5]:
def get_steps(train_data, test_data,val_data,batch_size):
    length=train_data.shape[0]
    if length % batch_size==0:
        tr_steps=int(length/batch_size)
    else:
        tr_steps=int(length/batch_size) + 1
    length=val_data.shape[0]
    if length % batch_size==0:
        v_steps=int(length/batch_size)
    else:
        v_steps=int(length/batch_size) + 1
    length=test_data.shape[0]
    batches=[int(length/n) for n in range(1,length+1) if length % n ==0 and length/n<=80]
    batches.sort(reverse=True)
    t_batch_size=batches[0]
    t_steps=length/t_batch_size        
    return [tr_steps,t_steps, v_steps, t_batch_size]

In [6]:
def make_model(source_dir,output_dit,class_list, image_size, subject,model_size, rand_seed,lr_factor):
    size=len(class_list)
    check_file = os.path.join(output_dir, 'tmp.h5')
        
    if model_size=='L':
        # mobile = keras.applications.mobilenet_v2.MobileNetV2(input_shape=input_shape)
        mobile = tf.keras.applications.mobilenet.MobileNet()        
        #remove last 5 layers of model and add dense layer with 128 nodes and the prediction layer with size nodes
        # where size=number of classes
        x=mobile.layers[-6].output
        x=Dense(128, kernel_regularizer = regularizers.l2(l = 0.015), activation='relu')(x)
        x=Dropout(rate=.5, seed=rand_seed)(x)
        predictions=Dense (size, activation='softmax')(x)
        model = Model(inputs=mobile.input, outputs=predictions)
        for layer in model.layers:
            layer.trainable=True
        model.compile(Adam(lr=lr_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
    else:
        if model_size=='M':
            fm=2
        else:
            fm=1
        model = Sequential()
        model.add(Conv2D(filters = 4*fm, kernel_size = (3, 3), activation ='relu', padding ='same', name = 'L11',
                         kernel_regularizer = regularizers.l2(l = 0.015),input_shape = (image_size, image_size, 3)))
        model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name ='L12'))
        model.add(BatchNormalization(name = 'L13'))
        model.add(Conv2D(filters = 8*fm, kernel_size = (3, 3), activation ='relu',
                         kernel_regularizer = regularizers.l2(l = 0.015), padding ='same', name = 'L21')) 
        model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name ='L22'))
        model.add(BatchNormalization(name = 'L23'))
        model.add(Conv2D(filters = 16*fm, kernel_size = (3, 3), activation ='relu',
                         kernel_regularizer = regularizers.l2(l = 0.015), padding ='same', name ='L31')) 
        model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name ='L32'))
        model.add(BatchNormalization(name = 'L33'))
        if fm==2:
            model.add(Conv2D(filters = 32*fm, kernel_size = (3, 3), activation ='relu',
                             kernel_regularizer = regularizers.l2(l = 0.015),padding ='same', name ='L41')) 
            model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name ='L42'))
            model.add(BatchNormalization(name = 'L43'))
            model.add(Conv2D(filters = 64*fm, kernel_size = (3, 3), activation ='relu', 
                             kernel_regularizer = regularizers.l2(l = 0.015),padding ='same', name ='L51')) 
            model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name ='L52'))
            model.add(BatchNormalization(name = 'L53'))
            
        model.add(Flatten())
        model.add(Dense(256 *fm,kernel_regularizer = regularizers.l2(l = 0.015), activation='relu', name ='Dn1'))
        model.add(Dropout(rate=.5))
        model.add(Dense(size, activation = 'softmax', name ='predict'))
        model.compile(Adam(lr=lr_rate),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=10, mode='min', verbose=1)
    checkpoint = ModelCheckpoint(check_file, monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=1)
    lrck=keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=lr_factor, patience=1,
                                           verbose=1, mode='min', min_delta=0.000001, cooldown=1, min_lr=1.0e-08)
    callbacks=[checkpoint,lrck, early_stop]
    return [model, callbacks]

In [7]:
def make_generators(data_sets, batch_size,t_batch_size,rand_seed):
    #data_set[0]=train data,[1]train labels,[2]=test data,[3]=test labels,[4]=value data,[5]=val labels,[6]=test files
    train_datagen=ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input,
                             horizontal_flip=True,
                             samplewise_center=True,
                             samplewise_std_normalization=True)
    train_gen=train_datagen.flow(data_sets[0],data_sets[1], batch_size=batch_size, seed=rand_seed)
    val_datagen=ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input,
                             samplewise_center=True,
                             samplewise_std_normalization=True)
    val_gen=val_datagen.flow(data_sets[4], data_sets[5], batch_size=batch_size, seed=rand_seed)
    test_datagen=ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input,
                             samplewise_center=True,
                             samplewise_std_normalization=True)
    test_gen=test_datagen.flow(data_sets[2], data_sets[3], batch_size=t_batch_size, shuffle=False)
    return [train_gen, test_gen, val_gen]

In [8]:
def train(model, train_gen, val_gen,tr_steps,v_steps, epochs,callbacks,sub):
    start=time.time()
    data = model.fit_generator(generator = train_gen, validation_data= val_gen,
                       steps_per_epoch=tr_steps, epochs=epochs, 
                       validation_steps=v_steps, callbacks = callbacks)
    stop=time.time()
    duration = stop-start
    hrs=int(duration/3600)
    mins=int((duration-hrs*3600)/60)
    secs= duration-hrs*3600-mins*60
    msg='The training cycle took  {0} hours {1} minutes and {2:6.2f} seconds'
    print(msg.format(hrs, mins,secs))
    return data
    

In [9]:
def tr_plot(tacc,vacc,tloss,vloss):
    #Plot the training and validation data
    Epoch_count=len(tloss)
    Epochs=[]
    for i in range (0,Epoch_count):
        Epochs.append(i+1)
    fig,axes=plt.subplots(nrows=1, ncols=2, figsize=(15,5))
    axes[0].plot(Epochs,tloss, 'r', label='Training loss')
    axes[0].plot(Epochs,vloss,'g',label='Validation loss' )
    axes[0].set_title('Training and Validation Loss')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[1].plot (Epochs,tacc,'r',label= 'Training Accuracy')
    axes[1].plot (Epochs,vacc,'g',label= 'Validation Accuracy')
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    plt.tight_layout
    plt.style.use('fivethirtyeight')
    plt.show()
 

In [10]:
def display_pred(source_dir,output_dir,pred,t_files,t_labels,class_list,subject, model_size):
    # t_files are the test files, t_labels are the class label associated with the test file
    # class_list is a list of classes
    trials=len(t_files)   # number of predictions made should be same as len(t_files)
    errors=0
    prob_list=[]
    true_class=[]
    pred_class=[]
    file_list=[]
    x_list=[]
    index_list=[]
    pr_list=[]
    error_msg=''
    for i in range (0,trials):
        p_c_num=pred[i].argmax()  #the index with the highest prediction value
        if p_c_num !=t_labels[i]: #if the predicted class is not the same as the test label it is an error
            errors=errors + 1
            file_list.append(t_files[i])  # list of file names that are in error
            true_class.append(class_list[t_labels[i]]) # list classes that have an eror
            pred_class.append(class_list[p_c_num]) #class the prediction selected
            prob_list.append(100 *pred[i][p_c_num])# probability of the predicted class
            add_msg='{0:^24s}{1:5s}{2:^20s}\n'.format(class_list[t_labels[i]], ' ', t_files[i])
            error_msg=error_msg + add_msg
            
    accuracy=100*(trials-errors)/trials
    print('\n There were {0} errors in {1} trials for an accuracy of {2:7.3f}'.format(errors, trials,accuracy,),flush=True)
    if errors<=25:
        msg='{0}{1:^24s}{0:3s}{2:^20s}{0:3s}{3:20s}{0:3s}{4}'
        print(msg.format(' ', 'File Name', 'True Class', 'Predicted Class', 'Probability'))
        for i in range(0,errors):
            msg='{0}{1:^24s}{0:3s}{2:^20s}{0:3s}{3:20s}{0:5s}{4:^6.2f}'
            print (msg.format(' ',file_list[i], true_class[i], pred_class[i], prob_list[i]))
    else:
        print('with {0} errors the full error list will not be printed'.format(errors))    
    acc='{0:6.2f}'.format(accuracy)
    if model_size=='L':
        ms='Large'
    elif model_size=='M':
        ms= 'Medium'
    else:
        ms= 'Small'
    header='Classification subject: {0} There were {1} errors in {2} tests for an accuracy of {3} using a {4} model\n'.format(subject,errors,trials,acc,ms)
    header= header +'{0:^24s}{1:5s}{2:^20s}\n'.format('CLASS',' ', 'FILENAME') 
    error_msg=header + error_msg
    file_name='error list-' + model_size + acc +'.txt'
    print('\n file {0} containing the list of errors has been saved to {1}'.format(file_name, output_dir))
    file_path=os.path.join(output_dir,file_name)
    f=open(file_path, 'w')
    f.write(error_msg)
    f.close()
    for c in class_list:
        count=true_class.count(c)
        x_list.append(count)
        pr_list.append(c)
    for i in range(0, len(x_list)):  # only plot classes that have errors
        if x_list[i]==0:
            index_list.append(i)
    for i in sorted(index_list, reverse=True):  # delete classes with no errors
        del x_list[i]
        del pr_list[i]      # use pr_list - can't change class_list must keep it fixed
    fig=plt.figure()
    fig.set_figheight(len(pr_list)/4)
    fig.set_figwidth(6)
    plt.style.use('fivethirtyeight')
    for i in range(0, len(pr_list)):
        c=pr_list[i]
        x=x_list[i]
        plt.barh(c, x, )
        plt.title('Errors by class')
    plt.show()
    time.sleep(5.0)
    
    return accuracy        

In [11]:
def save_model(output_dir,subject, accuracy,r_model, image_size):
    # save the model with the  subect-accuracy.h5
    acc=str(accuracy)[0:5]
    tempstr=subject + '-' +str(image_size) + '-' + acc + '.h5'
    model_save_path=os.path.join(output_dir,tempstr)
    r_model.save(model_save_path)    

In [12]:
def make_predictions(output_dir, test_gen, t_steps):
    # the best model was saved as a file need to read it in and load it since it is not available otherwise
    test_gen.reset()
    msg='Training has completed, now loading saved best model and processing test set to see how accurate the model is'
    print (msg,flush=True)
    model_path=os.path.join(output_dir,'tmp.h5')
    model=load_model(model_path)                      # load the saved model with lowest validation loss
    pred=model.predict_generator(test_gen, steps=t_steps,verbose=1) # make predictions on the test set
    return [pred, model]

In [13]:
def wrapup (source_dir,output_dir,subject, accuracy, r_model,image_size,run_num, model_size):
    if accuracy >= 95:
        msg='With an accuracy of {0:5.2f} the results appear satisfactory and the program will terminate'
        print(msg.format(accuracy),flush=True)
        return [False, None]
    elif accuracy >=85 and accuracy < 95:
        if run_num<2:
            msg='With an accuracy of {0:5.2f} our results are mediocure, will run 10 more epochs to see if accuracy improves '
            print (msg.format(accuracy),flush=True)
            return [True, 10]
        else:
            print('Final accuracy of {0} is still mediocure- program terminating'.format(accuracy))
            if model_size !='L':
                print('try running again with model_size=L to get a more accurate result')
            return [False, None]
    else:
        if run_num<2:
            msg='With an accuracy  of {0:5.2f} the results are poor, will run for 15 more epochs to see if accuracy improves'
            print (msg.format( accuracy),flush=True)
            msg='if running more epochs does not improve test set accuracy you may want to get more training data '
            msg=msg + 'or perhaps crop your images so the desired subject takes up most of the image'
            print (msg, flush=True)
            return [True, 15]
        else:
            print('Final accuracy of {0} is still poor - program is terminating'.format(accuracy))
            if model_size !='L':
                print('try running again with model_size=L to get a more accurate result')
            return [False, None] 

In [14]:
def TF2_classify(source_dir,output_dir,mode,subject, t_split=10, v_split=5, epochs=20,batch_size=80,
                 lr_rate=.002,lr_factor=.8,image_size=224,rand_seed=128,model_size='L'):
    model_size=model_size.upper()
    mode=mode.lower()
    if model_size=='L':
        image_size=224              # for the large model image size must be 224
    data_sets=get_data_sets(source_dir,output_dir,mode,subject,image_size,t_split, v_split, rand_seed)
    #data_set[0]=train data,[1]train labels,[2]=test data,[3]=test labels,[4]=value data,[5]=val labels,[6]=test files
    # data_sets[7]=class_list
    steps=get_steps(data_sets[0],data_sets[2],data_sets[4],batch_size)
    # tr_steps=steps[0]  t_steps=steps[1]   v_steps=steps[2] t_batch_size=steps[3]
    model_data=make_model(source_dir,output_dir,data_sets[7], image_size, subject, model_size,rand_seed,lr_factor)
    # model=model_data[0]  callbacks=model_data[1]
    gens=make_generators(data_sets,batch_size,steps[3], rand_seed)
    # train_gen = gens[0]  test_gen=gens[1]   val_gen=gens[2]
    run_num=0
    run=True
    tacc=[]
    tloss=[]
    vacc=[]
    vloss=[]
    while run:
        run_num=run_num +1
        results=train(model_data[0], gens[0], gens[2],steps[0],steps[2], epochs,model_data[1],subject)
        tacc_new=results.history['accuracy']
        tloss_new=results.history['loss']
        vacc_new =results.history['val_accuracy']
        vloss_new=results.history['val_loss']
        for d in tacc_new:  # need to append new data from training to plot all epochs
            tacc.append(d)
        for d in tloss_new:
            tloss.append(d)
        for d in vacc_new:
            vacc.append(d)
        for d in vloss_new:
            vloss.append(d)
        tr_plot(tacc,vacc,tloss,vloss)
        predict=make_predictions(output_dir, gens[1], steps[1],)
        # pred= predict[0]  r_model=predict[1]
        accuracy=display_pred(source_dir,output_dir,predict[0],data_sets[6],data_sets[3],data_sets[7], subject, model_size)
        decide=wrapup(source_dir,output_dir,subject, accuracy, predict[1], image_size,run_num, model_size)
        run=decide[0]
        epochs=decide[1] 

In [21]:
import os
os.getcwd()
d_list=os.listdir('./dog_classes2')
print(d_list)
for d in d_list:
    #print(d)
    d_path=os.path.join('./dog_classes2', d)
    print(d_path)
    class_list=os.listdir(d_path)
    for c in class_list:
        class_path=os.path.join(d_path, c)    
        print(c)
source_dir=class_path
output_dir='./working2'
subject='dog_breeds'
t_split=8
v_split=8
epochs=20
batch_size=40
lr_rate=.0015
lr_factor=.8
image_size=224
rand_seed=256
model_size='L'
mode='sep'

TF2_classify(source_dir,output_dir,mode,subject, t_split=t_split, v_split=v_split, epochs=epochs,batch_size=batch_size,
         lr_rate=lr_rate,lr_factor=lr_factor,image_size=image_size,rand_seed=rand_seed, model_size=model_size)

.-dog_classes2\images\image\train\10_WELSH_CORGI:  16%|████▌                       | 11/67 [00:00<00:00, 101.19files/s]

['images']
./dog_classes2\images
image
./dog_classes2\images\image\train\10_WELSH_CORGI\0.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\1.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\10.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\11.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\12.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\13.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\14.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\15.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\16.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\17.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\18.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\19.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\2.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\20.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\21.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\22.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\23.jpg
./dog_classe

.-dog_classes2\images\image\train\10_WELSH_CORGI:  55%|███████████████▍            | 37/67 [00:00<00:00, 112.48files/s]


./dog_classes2\images\image\train\10_WELSH_CORGI\25.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\26.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\27.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\28.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\29.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\3.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\30.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\31.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\32.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\33.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\34.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\35.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\36.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\37.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\38.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\39.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\4.jpg
./dog_classes2\images\image\train\10_WELSH_CORGI\

.-dog_classes2\images\image\train\10_WELSH_CORGI:  91%|█████████████████████████▍  | 61/67 [00:00<00:00, 110.99files/s]

...


./dog_classes2\images\image\valid\X_SHIH_TZU\88.jpg
./dog_classes2\images\image\valid\X_SHIH_TZU\89.jpg
./dog_classes2\images\image\valid\X_SHIH_TZU\90.jpg
./dog_classes2\images\image\valid\X_SHIH_TZU\91.jpg
./dog_classes2\images\image\valid\X_SHIH_TZU\92.jpg
./dog_classes2\images\image\valid\X_SHIH_TZU\93.jpg
./dog_classes2\images\image\valid\X_SHIH_TZU\94.jpg
./dog_classes2\images\image\valid\X_SHIH_TZU\95.jpg
./dog_classes2\images\image\valid\X_SHIH_TZU\96.jpg
./dog_classes2\images\image\valid\X_SHIH_TZU\97.jpg


./dog_classes2\images\image\valid\X_SHIH_TZU\98.jpg
./dog_classes2\images\image\valid\X_SHIH_TZU\99.jpg
[1, 5, 14, 2, 19, 15, 11, 18, 12, 19, 8, 1, 16, 13, 13, 11, 0, 7, 3, 2, 14, 6, 0, 16, 13, 6, 11, 15, 16, 8, 6, 7, 19, 13, 9, 16, 16, 6, 5, 19, 15, 18, 17, 6, 13, 10, 8, 6, 6, 6, 9, 19, 15, 6, 15, 14, 16, 14, 9, 3, 7, 4, 3, 9, 6, 5, 11, 4, 14, 19, 19, 11, 16, 19, 15, 19, 9, 16, 13, 1, 1, 1, 9, 13, 5, 5, 0, 13, 15, 11, 8, 12, 5, 14, 14, 12, 19, 15, 10, 19, 6, 14, 2, 3, 17, 17, 14, 6, 4, 19, 10, 8, 17, 11, 9, 13, 11, 19, 13, 19, 16, 8, 11, 9, 16, 15, 4, 0, 19, 3, 3, 17, 13, 6, 1, 14, 18, 16, 7, 10, 18, 7, 10, 1, 7, 6, 6, 18, 11, 4, 6, 8, 18, 17, 6, 9, 6, 9, 4, 9, 9, 19, 9, 12, 12, 18, 18, 15, 18, 19, 4, 6, 10, 16, 5, 19, 1, 16, 11, 3, 5, 1, 14, 19, 9, 1, 13, 18, 11, 3, 17, 16, 8, 9, 11, 11, 5, 12, 11, 19, 9, 13, 3, 4, 6, 19, 1, 15, 18, 6, 18, 9, 19, 2, 14, 1, 9, 6, 14, 2, 18, 4, 18, 11, 15, 5, 6, 19, 17, 11, 10, 6, 5, 19, 8, 3, 4, 5, 1, 8, 14, 7, 6, 6, 4, 11, 8, 6, 4, 16, 13, 16, 11, 11

W0804 19:46:44.196849 14100 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.
W0804 19:46:48.914114 14100 deprecation.py:317] From <ipython-input-8-a768e171d41f>:3: Model.fit_generator (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
Please use Model.fit, which supports generators.


Epoch 1/20


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/conv1/Conv2D (defined at <ipython-input-8-a768e171d41f>:3) ]] [Op:__inference_train_function_8225]

Function call stack:
train_function


In [19]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [593]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15097798094400132463
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 18075146008070931256
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1414335283
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14637219652679307152
physical_device_desc: "device: 0, name: GeForce MX150, pci bus id: 0000:02:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9361542662056277506
physical_device_desc: "device: XLA_GPU device"
]


In [614]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [611]:
import tensorflow as tf
tf.test.is_gpu_available()
tf.test.is_built_with_cuda()

True

In [736]:
tf.debugging.set_log_device_placement(True)

# 텐서를 CPU에 할당
with tf.device('/device:XLA_GPU:0'):
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

c = tf.matmul(a, b)
print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [747]:
!nvidia-smi

Tue Aug 04 14:17:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 445.87       Driver Version: 445.87       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce MX150      WDDM  | 00000000:02:00.0 Off |                  N/A |
| N/A   57C    P0    N/A /  N/A |   1573MiB /  2048MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  